# 使用ResNet50搭建cifar-10分类系统

In [4]:
import numpy as np
import pandas as pd
import os

In [15]:
#首先将各个类别的图片和种类进行对应（训练集）
with open('train.txt','w') as w:
    for root, dirs, files in os.walk('train/'):
        for i in files:
            if i[0]==('.'):
                continue
            if i.split('_')[2] == 'automobile':
                w.write(i+' '+'0\n')
            elif i.split('_')[2] == 'airplane':
                w.write(i+' '+'1\n')
            elif i.split('_')[2] == 'bird':
                w.write(i+' '+'2\n')
            elif i.split('_')[2] == 'cat':
                w.write(i+' '+'3\n')
            elif i.split('_')[2] == 'deer':
                w.write(i+' '+'4\n')
            elif i.split('_')[2] == 'frog':
                w.write(i+' '+'5\n')
            elif i.split('_')[2] == 'dog':
                w.write(i+' '+'6\n')
            elif i.split('_')[2] == 'truck':
                w.write(i+' '+'7\n')
            elif i.split('_')[2] == 'ship':
                w.write(i+' '+'8\n')
            elif i.split('_')[2] == 'horse':
                w.write(i+' '+'9\n')

In [16]:
#首先将各个类别的图片和种类进行对应（测试集）
with open('test.txt','w') as w:
    for root, dirs, files in os.walk('test/'):
        for i in files:
            if i[0]==('.'):
                continue
            if i.split('_')[2] == 'automobile':
                w.write(i+' '+'0\n')
            elif i.split('_')[2] == 'airplane':
                w.write(i+' '+'1\n')
            elif i.split('_')[2] == 'bird':
                w.write(i+' '+'2\n')
            elif i.split('_')[2] == 'cat':
                w.write(i+' '+'3\n')
            elif i.split('_')[2] == 'deer':
                w.write(i+' '+'4\n')
            elif i.split('_')[2] == 'frog':
                w.write(i+' '+'5\n')
            elif i.split('_')[2] == 'dog':
                w.write(i+' '+'6\n')
            elif i.split('_')[2] == 'truck':
                w.write(i+' '+'7\n')
            elif i.split('_')[2] == 'ship':
                w.write(i+' '+'8\n')
            elif i.split('_')[2] == 'horse':
                w.write(i+' '+'9\n')

In [36]:
#导入相应的包
import cv2
import time
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.utils import np_utils
from keras.applications.resnet50 import ResNet50
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [37]:
#把txt转换为csv
def tranverse_images(path):
    labels = pd.DataFrame()
    flowers = []
    items = []
    with open(path,'r') as f:
        for line in f.readlines():
            flower,item = line.strip().split(' ')
            flowers.append(flower)
            items.append(item)
    labels_data = pd.DataFrame({'flower': flowers, 'labels': items})
    labels = pd.concat((labels, labels_data)) 
    return labels

train_and_labels = tranverse_images('train.txt')#50000行数据
test_and_labels = tranverse_images('test.txt')#10000行数据

In [38]:
#查看图片尺寸
image = cv2.imread('./train/batch_1_airplane_29.jpg')
image.shape#(32, 32, 3)每张图片的size如此
image = cv2.imread('./test/batch_1_airplane_2679.jpg')
image.shape#(32, 32, 3)每张图片的size如此

(32, 32, 3)

In [39]:
#img2array
def img2array(label,path):
    lis_imgs = [I for I in train_and_labels['flower']]
    return np.array([np.array(Image.open(path+img)) for img in lis_imgs])

X = img2array(train_and_labels,'./train/')
print(X.shape)
np.save('./X_train.npy',X)

(50000, 32, 32, 3)


In [40]:
def img2array2(label,path):
    lis_imgs = [I for I in test_and_labels['flower']]
    return np.array([np.array(Image.open(path+img)) for img in lis_imgs])

test_X = img2array2(test_and_labels,'./test/')
print(test_X.shape)
np.save('./X_test.npy',test_X)

(10000, 32, 32, 3)


In [41]:
X = X/255
test_X = test_X/255
y_train = train_and_labels['labels']
y_test = test_and_labels['labels']
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [44]:
def flower_model(X_train, y_train):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
    for layers in base_model.layers:
        layers.trainable = False
    model = Flatten()(base_model.output)
    model = Dense(128, activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(10, activation='softmax')(model)

    model = Model(inputs=base_model.input, outputs=model)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=50) 

    return model

In [45]:
model = flower_model(X, y_train)
model.evaluate(test_X, y_test, verbose=0)

/Users/caijie/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/50
50000/50000 [==============================] - 322s 6ms/step - loss: 1.8774 - acc: 0.3451
Epoch 2/50
50000/50000 [==============================] - 324s 6ms/step - loss: 1.6031 - acc: 0.4351
Epoch 3/50
50000/50000 [==============================] - 317s 6ms/step - loss: 1.5125 - acc: 0.4644
Epoch 4/50
50000/50000 [==============================] - 310s 6ms/step - loss: 1.4666 - acc: 0.4834
Epoch 5/50
50000/50000 [==============================] - 311s 6ms/step - loss: 1.4236 - acc: 0.4951
Epoch 6/50
50000/50000 [==============================] - 301s 6ms/step - loss: 1.3891 - acc: 0.5105
Epoch 7/50
50000/50000 [==============================] - 322s 6ms/step - loss: 1.3616 - acc: 0.5167
Epoch 8/50
50000/50000 [==============================] - 314s 6ms/step - loss: 1.3497 - acc: 0.5222
Epoch 9/50
50000/50000 [==============================] - 335s 7ms/step - loss: 1.3243 - acc: 0.5294
Epoch 10/50
50000/50000 [==============================] - 321s 6ms/step - loss: 1.3117 - a

[11.559770764160156, 0.10000000000000001]